# Python Bootcamp Day 2 Morning

* Instructor:  Andrew Yarmola [andrew.yarmola@gmail.com](mailto:andrew.yarmola@gmail.com)
* Bootcamp files: [github.com/andrew-yarmola/python-bootcamp](https://github.com/andrew-yarmola/python-bootcamp)


## Comments

### Classes as decorators

Because class objects are **callable** (usually to instantiate an object), they can also be used as decorators. Sometimes this is more natural, but you have to manage the reference to `func` and any other extra argumetns yourself without the automatic benefit (or confusion) of closures. For example

In [ ]:
import functools

class CountCalls:
    def __init__(self, func):
        # runs @wraps on the self instance
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)
        
@CountCalls
def do_something() :
    print("did a lot")

do_something()
do_something()

### Decorating a class

Like function, classes constructors respond to a `.__call__()` method, so we can wrap instance construction in decorator just like functions.

In [ ]:
from bootcamp_decorators import time_this

# decorates instantiation only
# uses: debug or inspect class attibures
@time_this 
class Employee:
    def __init__(self, id, name):
        self.id = id
        self.name = name
        
    # undecorated instance method
    def calculate_payroll(self):
        pass

### `functools` and `dataclass` decorators

Be aware, that many modules provide decorators you might already want to use. For example,

In [ ]:
import functools

# this decorator caches up to 4 return values at a time
@functools.lru_cache(maxsize=4)
def fibonacci(num) :
    print(f"Calculating fibonacci({num})")
    if num < 2 :
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

The `dataclass` decorator below allows you to save time writing methods like `__init__`, `__repr__` for classes that only store data and have a few instance methods.

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True) # <--- make an immutable dataclass
class NamedPoint:
    name: str
    x : float = 0.0 # <--- type annotation with default values
    y : float = 0.0

    def distance_to(self, other):
        return ((self.x - other.x)**2 + (self.y - other.y)**2)**0.5

In [ ]:
a = NamedPoint('a', 1., 2.)
b = NamedPoint('b', 0., 2.)
a.distance_to(b)

In [ ]:
a.x = 7

## Writing Packages

Packages are simply a collection of modules structured in a way to allow for “dotted module names”. For example, the module `sound.formats` corresponds to a submodule `formats` of package named `sound`. This helps control namespaces, so someone working on `formats` does not need to know what the `sound` namespace looks like.

Suppose you want to design a collection of modules handling of sound files and sound data. There are many different sound file formats, so you may need to create and maintain modules for the conversion between the various file formats. You will also want to add modules for different audio processing tasks.

Here is a possible structure for your package :

```
/home/user/sound                Repo-level
├── README.md
├── Pipfile
├── Pipfile.lock
├── sound/                      Top-level package
│   ├── __init__.py             Initialize the sound package
│   ├── formats/                Subpackage for file format conversions
│   │   ├── _init__.py
│   │   ├── wavread.py
│   │   ├── wavwrite.py
│   │   └── ...
│   ├── effects/                Subpackage for sound effects
│   │   ├── __init__.py
│   │   ├── echo.py
│   │   ├── surround.py
│   │   └── ...
│   └── filters/                  Subpackage for filters
│       ├── __init__.py
│       ├── equalizer.py
│       ├── karaoke.py
│       └── ...
└── tests/                      Tests directory
    ├── test_filters.py
    └── ...
```

When importing the package, Python searches through the directories in `sys.path` looking for the package subdirectory.

The `__init__.py` files are required to make python treat directories containing the file as packages. This prevents directories with a common name unintentionally hiding valid modules that occur later on the module search path.

One can import submodules via 

```python
import sound.effects.echo
```

In the simplest case, `__init__.py` can just be an empty file. But you can also execute initialization code in `__init__.py` for when your package is imported. For example, the line

```python
__all__ = ["effects", "filters"]
```

will force the `from sounds import formats` to fail and only succeed with `effects` and `filters`.

Other options include things like
```python
import sound.filters as fltrs
from .formats import wav_read
````
where inside the formats module `__init__.py` we have
```python
import sound.filters as fltrs
````

Let's take a look at the `sound` module example included in the repository.

**Upshot**. You should use `__init__.py` to setup the namespace of your package when it is imported. If you don't need anything extra, you can leave it blank.

**More about imports**. You can read more about the structure of `import` at [realpython.com/python-modules-packages/](https://realpython.com/python-modules-packages/)

**Remark**. There is really not fully agreed upon format to what to put in `__init__.py`. See the following post for a discussion on the matter [pcarleton.com/2016/09/06/python-init/](https://pcarleton.com/2016/09/06/python-init/)

### A Flask app layout

A Flask app would have relatively similar layout to package. Again, tests are usually kept outside of the application package to avoid access to them during runtime.

```
/home/user/flaskr
├── run_app.py
├── README.md
├── Pipfile
├── Pipfile.lock
├── flaskr/
│   ├── __init__.py
│   ├── db.py
│   ├── schema.sql
│   ├── auth.py
│   ├── blog.py
│   ├── templates/
│   │   ├── base.html
│   │   ├── auth/
│   │   │   ├── login.html
│   │   │   └── register.html
│   │   └── blog/
│   │       ├── create.html
│   │       ├── index.html
│   │       └── update.html
│   └── static/
│       └── style.css
└── tests/
    ├── conftest.py
    ├── data.sql
    ├── test_factory.py
    ├── test_db.py
    ├── test_auth.py
    └── test_blog.py
```

### Installable packages

Installable packaged that are hosted in repositories are also an option. This requires becoming familiar with `distutils`, `setuptools`, and `setup.py` and `MANIFEST.in` formatting. See [realpython.com/pypi-publish-python-package/](https://realpython.com/pypi-publish-python-package/) and [docs.python-guide.org/writing/structure/](https://docs.python-guide.org/writing/structure/) for guides and comments.


## Managing several repositories

Since you will  most likely not be creating installable and distributed packages, I would suggest you use `git submodule` to track all of your internal dependencies. For external dependencies, use `pipenv`

To use `import`, you will need to add the root module directory to your path using `sys.path.append()` as necessary.

If you really need to install a package using `pipenv` form a `git` repository, this can be done with

```bash
pipenv install -e git+https://github.com/requests/requests.git@v2.20.1#egg=requests
```

where `#egg=package_name` specifies the package name used by `pipenv`

## Scripts

Crating scripts in python is very easy. You simply have to check the `__name__` variable of your module.

For example, here are the contents of `three_powers_of_two.py` : a script that prints the first 3 powers of 2.

```python
#!/usr/bin/env python3

def generate_powers() :
    return [ 2**x for x in range(3) ]

print("Global __name__ is :", __name__)

if __name__ == '__main__' :
    print(*generate_powers(), sep = '\n')
```

Now, if I import this file, nothing will happend except for the fact that I will have `generate_powers` defined.

In [ ]:
import three_powers_of_two as tpt

As you can see, when importing, the global variable `__name__` is set to the filename.

However, if I now go to a console/terminal and **run** the scipt using
```bash
python3 three_powers_of_two.py
```
or, because we have `#!/usr/bin/env python3` at the top of the file, we can use
```bash
./three_powers_of_two.py
```

You will see the commands in the `if` statement executed.

```bash
$ ./three_powers_of_two.py 
Global __name__ is : __main__
1
2
4
```

So a script can be used both as a module and a tool. However, a program isn't very useful is you can't give it input.

### Arguments and Options

The standard way to pass arguments to a script is to give a space separated list after the command call :

```bash
python three_powers_of_two.py arg1 arg2
```

To read the aguemnts in, we will use the `sys` module's `sys.argv` attribute. We update our `three_powers_of_two_new.py` with : 

```python
#!/usr/bin/env python3

import sys

def generate_powers() :
    return [ 2**x for x in range(3) ]

if __name__ == '__main__' :

    print(sys.argv)
    
    print(*generate_powers(), sep = '\n')
```

When we run this using the above command in a terminal, we will see :
```bash
$ python three_powers_of_two.py arg1 arg2
['three_powers_of_two.py', 'arg1', 'arg2']
1
2
4
```
In particular, `sys.argv` is a list that starts with the **name** of the program and then gives **all space separated arugments**.

**Remark** if you need to have a space in an arugment, you can use (double) quatition marks :
```bash
$ python three_powers_of_two.py arg1 "arg2 with a space"
['three_powers_of_two.py', 'arg1', 'arg2 with a space']
1
2
4
```

Let us make a slightly more useful script `count_vowels.py` that counts vowels in a file

```python
#!/usr/bin/env python3

import sys

def vowels_in_string(data) :
    return { v : data.count(v) for v in 'aeiou' }

if __name__ == '__main__' :
    if len(sys.argv) < 2 :
        print("Usage: python count_vowels.py file")
        sys.exit(2)
        
    file_name = sys.argv[1]

    with open(file_name, 'r') as fp :
        data = fp.read()
        print(vowels_in_string(data))
```

Notice that I am doing some minimal input checking. This allows me to both inform the user how to use the program and also to check for bad input.

We can run out program to get :
```bash
$ python count_vowels.py "morning.ipynb"
{'u': 5197, 'a': 5898, 'i': 5305, 'o': 4934, 'e': 6094}
```

### Options and `getopt`

Using `sys.argv` gives us only **positional** arguments for our program. There is a better way using the `getopt` modules. The idea is to specify a **flag** or **keyword** using a `-` or `--` prefix. We would like to do something like this : 

```bash
./hanoi_gif.py -v --d 4 --fps 4 awesome_hanoi_4.gif
```

Let's see a simple example fo how `getopt` works

In [ ]:
import getopt

argv_list = '-v -d 4 --fps 5 -w something --write nothing arg1 arg2 arg3'.split()

opts, args = getopt.getopt(argv_list, 'vd:w:',
                           ['verbose', 'disk=', 'fps=', 'write='])

print(opts)
print(args)

As you can see there are **three** types of options/arguments here. The key thing to understand is the line
```python
opts, args = getopt.getopt(argv_list, 'vd:w:',
                           ['verbose', 'disk=', 'fps=', 'write='])
```
The string `'vd:w:'` indicates how to parse **single letter** options preceded by a `-` symbol. While the list `['verbose', 'disk=', 'fps=', 'write=']` indicates how to parse **keyword** options preceded by a `--` symbol

The options here are :
* **flags** : these are the `-v` or `--verbose` options
   * they have no value, their **presence** is all you need
   * they are declared by a letter **without** a colon or a word **without** an `=`

* **keyword arguments** : these are the `-d`,`-w`, `--disks`, `--write`, and `--fps` options
   * they require a value to follow them when calling the program
   * their declaration is followed by a colon after a letter or an `=` after a keyword
   
* **positional arguments** : there are `arg1`, `arg2`, and `arg3`
   * must **follow** all flag and keyword arguments
   
When `getopt.getopt` parses `argv_list`, it returns a **list of tuples** and a **list of strings**. The list of tuples is map of options to their values and the list of strings is the list of positional arguments.

To apply this to function arguments, we simply need to call `getopt` on `sys.argv[1:]` (dropping the program name).

We can now implement a `hanoi_gif.py` containing the following code. Pay close attention to how I interpret the contents of `opts` and `args`.

In [ ]:
%cat hanoi_gif.py

## Flask

Flask is a lightweight web application framework that wraps the Werkzeug web application library and uses Jinja for it's template engine.

The Flask documentation is rather extensive and we will essentially go through the tutorial at [flask.palletsprojects.com](https://flask.palletsprojects.com) and some others.

### Werkzeug


Werkzeug is a comprehensive WSGI web application library. WSGI stands for Web Server Gateway Interface. It implements a specification that describes how a web server communicates with web applications, such as Flask app written in python. 

Flask wraps Werkzeug, using it to handle the details of WSGI while providing more structure and patterns for defining powerful applications.

Werkzeug includes:

* An interactive debugger that allows inspecting stack traces and source code in the browser with an interactive interpreter for any frame in the stack.
* A full-featured request object with objects to interact with headers, query args, form data, files, and cookies.
* A response object that can wrap other WSGI applications and handle streaming data.
* A routing system for matching URLs to endpoints and generating URLs for endpoints, with an extensible system for capturing variables from URLs.
* HTTP utilities to handle entity tags, cache control, dates, user agents, cookies, files, and more.
* A threaded WSGI server for use while developing applications locally.
* A test client for simulating HTTP requests during testing without requiring running a server.
Werkzeug is Unicode aware and doesn't enforce any dependencies. It is up to the developer to choose a template engine, database adapter, and even how to handle requests.

### Jinja

Jinja2 is a full-featured template engine for Python with full unicode support. A template engine is a system for dynamically generating HTML and other web formats.

Here is what templates look like.

```
{% extends "layout.html" %}
{% block body %}
  <ul>
  {% for user in users %}
    <li><a href="{{ user.url }}">{{ user.username }}</a></li>
  {% endfor %}
  </ul>
{% endblock %}
```

* Sandboxed execution mode. Every aspect of the template execution is monitored and explicitly whitelisted or blacklisted, whatever is preferred. This makes it possible to execute untrusted templates.
* powerful automatic HTML escaping system for cross site scripting prevention.
* Template inheritance makes it possible to use the same or a similar layout for all templates.
* High performance with just in time compilation to Python bytecode. Jinja2 will translate your template sources on first load into Python bytecode for best runtime performance.
* Optional ahead-of-time compilation
* Easy to debug with a debug system that integrates template compile and runtime errors into the standard Python traceback system.
* Configurable syntax. For instance you can reconfigure Jinja2 to better fit output formats such as LaTeX or JavaScript.
* Template designer helpers. Jinja2 ships with a wide range of useful little helpers that help solving common tasks in templates such as breaking up sequences of items into multiple columns and more.

```python
from flask import Flask, escape, request

app = Flask(__name__)

@app.route('/')
def index() :
    # the request object is a global variable name that has all the info
    # about the made request
    name = request.args.get("name", "World")
    return f'Hello, {escape(name)}!'
```

We can run this code from a shell using the `flask` shell command

```bash
export FLASK_APP=app.py
export export FLASK_ENV=development
flask run
```

Let's tale a look at [flask.palletsprojects.com/en/1.1.x/api/#flask.Request](https://flask.palletsprojects.com/en/1.1.x/api/#flask.Request)

The decorator wrapped function are called **view functions**. The return value from a view function is automatically converted into a response object for you. If the return value is a string it’s converted into a response object with the string as response body, a 200 OK status code and a text/html mimetype. If the return value is a `dict`, `jsonify()` is called to produce a response. The logic that Flask applies to converting return values into response objects is as follows:

* If a response object of the correct type is returned it’s directly returned from the view.
* If it’s a string, a response object is created with that data and the default parameters.
* If it’s a dict, a response object is created using jsonify.
* If a tuple is returned the items in the tuple can provide extra information. Such tuples have to be in the form (response, status), (response, headers), or (response, status, headers). The status value will override the status code and headers can be a list or dictionary of additional header values.
* If none of that works, Flask will assume the return value is a valid WSGI application and convert that into a response object.
* If you want to get hold of the resulting response object inside the view you can use the `make_response()` function.

### Variable Rules
You can add variable sections to a URL by marking sections with `<variable_name>`. Your function then receives the `<variable_name>` as a keyword argument. Optionally, you can use a converter to specify the type of the argument like `<converter:variable_name>`.

```python
@app.route('/user/<username>')
def user_profile(username):
    # show the user profile for that user
    return escape(f'User {username}')

@app.route('/post/<int:post_id>')
def post(post_id):
    # show the post with the given id, the id is an integer
    return escape(f'Post {post_id}')

@app.route('/path/<path:subpath>')
def path(subpath):
    # show the subpath after /path/
    return escape(f'Subpath {subpath}')

@app.route('/projects/')
def projects():
    return 'The project page'

@app.route('/about') # <---- different from /about/
def about():
    return 'The about page'
```
Converter types:

|||
|--- |--- |
|string|(default) accepts any text without a slash|
|int|accepts positive integers|
|float|accepts positive floating point values|
|path|like string but also accepts slashes|
|uuid|accepts UUID strings|


In [ ]:
from flask import Flask, escape, request

In [ ]:
help(escape)

If we want to look up the url for a given function call, there is the `url_for` function.

```python
from flask import url_for

with app.test_request_context():
    print("Got url for index :", url_for('index'))
    print("Got url for projects :", url_for('projects'))
    print("Got url for John : ", url_for('user_profile', username='John Doe'))
```

One can also use `url_for` to get the urls of static files

```python
url_for('static', filename='style.css')
```

#### GET and POST

The global `request` object can also track the method used to load the URL. Based on that, our code can perform different tasks.

```python
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        return "Logging you in"
    else:
        return "Here is your profile"
```

#### Basic Template Rendering

Let's add a file `welcome.html` into `templates/` with the content

```
<!doctype html>
<title>Welcome to Flask</title>
{% if name %}
  <h1>You say your name is {{ name }}!</h1>
{% else %}
  <h1>Hello, stranger!</h1>
{% endif %}
```

and add the following code to our app

```python
from flask import render_template, make_response

@app.route('/welcome/')
@app.route('/welcome/<name>')
def welcome(name = None):
    # can also just return render_template('welcome.html', name = name)
    resp = make_response(render_template('welcome.html', name = name))
    resp.set_cookie('name', name) # <-- you can access cookies from request.cookies
    return resp
```

Inside templates you also have access to the global `request`, `session` and `g` objects (the latter is the global namespace).

Automatic escaping is enabled, so if name contains HTML it will be escaped automatically. If you can trust a variable and you know that it will be safe HTML

#### Uploading

The `request` object takes care of file uploading for you.

```python
from werkzeug.utils import secure_filename

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        for fs in request.files.values() : # <-- .files is a Werkzeug MultiDict with FileStorage as values
            fs.save('/var/tmp/' + secure_filename(fs.filename))
```

#### Redirect and Errors

```python
from flask import abort, redirect, url_for

@app.route('/nope')
def index():
    return redirect(url_for('welcome'))

@app.route('/err')
def login():
    abort(401)
    print("I won't get to print")
```

#### Session

In addition to the `request` object there is also the `session` object which allows you to store information specific to a user from one request to the next. This is implemented on top of cookies and signs the cookies cryptographically. What this means is that the user could look at the contents of your cookie but not modify it, unless they know the secret key used for signing.

In order to use sessions you have to set a secret key. Here is how sessions work:

```python
from flask import Flask, session, redirect, url_for, escape, request

app = Flask(__name__)

# Set the secret key to some random bytes.
# Should be random and very secret
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

@app.route('/')
def index():
    if 'username' in session:
        return 'Logged in as %s'.format(escape(session['username']))
    return 'You are not logged in'

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        session['username'] = request.form['username']
        return redirect(url_for('index'))
    return '''
        <form method="post">
            <p><input type=text name=username>
            <p><input type=submit value=Login>
        </form>
    '''

@app.route('/logout')
def logout():
    # remove the username from the session if it's there
    session.pop('username', None)
    return redirect(url_for('index'))
```

#### Launching without `flask run`

You can always run you app with `pipenv run flask run` if you prefer not working in `pipenv shell`.

However, if you would like to avoid using `flask run`, you can make you `app.py` into a script by adding

```python
if __name__ == "__main__":
    app.run(host = 'localhost', port = 5000, debug = True)
```
at the end of the file.


#### Logging

Flask comes with preconfigured loggers, which are standard python `Logger` objects.

```python
app.logger.debug('A value for debugging')
app.logger.warning('A warning occurred (%d apples)', 42)
app.logger.error('An error occurred')
```

## Connexion, Flask-RESTPlus, and Flask-RESTful

These are three Flask extensions that help make writing REST api's much easier. Connexion is a much larger and more advanced framework that focuses on your api layout and linking it to python calls, while Flask-RESTPlus and Flask-RESTful are very much reliant on your code defining much of the api. Here is what they say about themselves

* Flask-RESTful/Flask-RESTPlus is an extension for Flask that adds support for quickly building REST APIs. Flask-RESTPlus encourages best practices with minimal setup. If you are familiar with Flask, Flask-RESTful/Flask-RESTPlus should be easy to pick up. Flask-RESTPlus provides a coherent collection of decorators and tools to describe your API and expose its documentation properly (using Swagger).

* Connexion is a framework that automagically handles HTTP requests based on OpenAPI Specification (formerly known as Swagger Spec) of your API described in YAML format. Connexion allows you to write an OpenAPI specification, then maps the endpoints to your Python functions; this makes it unique, as many tools generate the specification based on your Python code. You can describe your REST API in as much detail as you want; then Connexion guarantees that it will work as you specified.


Here is an example of Flask-RESTPlus, which is an extension of Flask-RESTful.

```python
from flask import Flask, request
from flask_restplus import Resource, Api

app = Flask(__name__)
api = Api(app)

todos = {}

@api.route('/<string:todo_id>')
class TodoSimple(Resource):
    def get(self, todo_id):
        return {todo_id: todos[todo_id]}

    def put(self, todo_id):
        todos[todo_id] = request.form['data']
        return {todo_id: todos[todo_id]}

if __name__ == '__main__':
    app.run(debug=True)
```

We will look at an example of a `connexion` project in detail a bit later.

### SQLAlchemy

SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL. SQLAlchemy is most famous for its object-relational mapper (ORM), an optional component that provides the data mapper pattern, where classes can be mapped to the database in open ended, multiple ways - allowing the object model and database schema to develop in a cleanly decoupled way from the beginning. The ORM mapper makes SQL injection attacks almost impossible.

Model object in SQLAlchemy very much resesble the structure of a table

```python
from datetime import datetime
from flask_sqlalchemy import SQLAlchemy

app = ...

db = SQLAlchemy(app)

class Person(db.Model):
    __tablename__ = "person"
    person_id = db.Column(db.Integer, primary_key=True)
    lname = db.Column(db.String(32))
    fname = db.Column(db.String(32))
    timestamp = db.Column(
        db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow
    )   
```

As you will see, an instance of person will have settable attributes. For example

```python
new_person = Person()
new_person.lname = "Yarmola"
new_person.timestamp  = datetime.utcow
db.session.add(new_person)
db.session.commit()
```

### Marshmallow

Marshmallow is an ORM/ODM/framework-agnostic library for converting complex datatypes, such as objects, to and from native Python datatypes. For a REST api, one would be intersted in serializing and deserializing Python object into JSON or similar fomats.

Here is an executable example

In [ ]:
from datetime import date
from marshmallow import Schema, fields
from pprint import pprint

class ArtistSchema(Schema):
    name = fields.Str()

class AlbumSchema(Schema):
    title = fields.Str()
    release_date = fields.Date()
    artist = fields.Nested(ArtistSchema())

class Artist :
    def __init__(self, name) :
        self.name = name
    
bowie = Artist('Bowie')
album = dict(artist=bowie, title='Hunky Dory', release_date=date(1971, 12, 17))

schema = AlbumSchema()
result = schema.dump(album)
pprint(result.data, indent=4)

### Python REST APIs With Flask, Connexion, and SQLAlchemy from Real Python

Let's go over the 3 webarchive guide for the `connexion-flask` project.